In [18]:
import csv
import os
import json
import codecs
import re
import pandas as pd

In [19]:
import xlrd

In [20]:
data = pd.read_excel('updated_bangla.xlsx')

In [21]:
data = list(data['text'])

In [22]:
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [23]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

['  ডেঙ্গু  ডোঙ্গল লিখেছেন   পোস্টটি করা হয়েছে বৃহস্পতিবার ০৮ জুলাই ২০১০ ০২২৪৩৩ পরিচ',
 '  ডেঙ্গু  ডোঙ্গল লিখেছেন আশাবাদী পোস্টটি করা হয়েছে শনিবার ১০ জুলাই ২০১০ ২৩২৪৫৯ ক্লাসিক্যাল ডেঙ্',
 '  আবার  ডেঙ্গু জ্বর  সতর্ক থাকুন  ',
 '24  ডেঙ্গু  আক্রান্ত যুবরাজকে ছাড়াই নামছে ভারত ',
 '   ডেঙ্গু জ্বরের সেবাযত্ন ডেঙ্গু কি ১ডেঙ্গু একটি ভাইরাসজনিত জ্বর এডিস মশার মাধ্যমে  ডেঙ্গু জ্বর  ছড়ায়',
 '   চারিদিকে প্রচুর  ডেঙ্গু  জ্বরের প্রকোপ বেড়েছে  ডেঙ্গু  কি ১ডেঙ্গু  একটি ভাইরাসজনিত জ্বর এডিস মশার মাধ্',
 '    ডেঙ্গু ',
 '   মশা দিয়ে মশা নিধন  স্ত্রী এডিস মশার কামড়েই  ডেঙ্গু জ্বর  হয়ে থাকে বিশ্ব স্বাস্থ্য সংস্থার হিসাবে ব',
 'ডেঙ্গু জ্বর  রুখতে বিজ্ঞানীদের সাফল্য ডেঙ্গু ভাইরাস প্রতিরোধে বিশ্বের নানা দেশে সফলতার খবর শোনা যায়৷ তারপরও মশা',
 'ডেঙ্গু জ্বর  রুখতে বিজ্ঞানীদের সাফল্য ডেঙ্গু ভাইরাস প্রতিরোধে বিশ্বের নানা দেশে সফলতার খবর শোনা যায়৷ তারপরও মশা',
 '    ডেঙ্গুজ্বর  নির্মূলের টিকা আসছে  ডেঙ্গুজ্বর  নির্মূলের টিকা আসছেএডিস মশার কামড় থেকে সৃষ্ট  ডেঙ্গুজ্বর ',
 '   বের হলো  ডেঙ্গু জ্বর  প্রতিরোধে 

In [52]:
data_words = data.copy()

In [58]:
str(data_words[3]).split(' ')

['24', '', 'ডেঙ্গু', '', 'আক্রান্ত', 'যুবরাজকে', 'ছাড়াই', 'নামছে', 'ভারত', '']

In [59]:
tokenized_data = [str(data_words[i]).split(' ') for i in range(len(data_words))]

In [60]:
tokenized_data

[['',
  '',
  'ডেঙ্গু',
  '',
  'ডোঙ্গল',
  'লিখেছেন',
  '',
  '',
  'পোস্টটি',
  'করা',
  'হয়েছে',
  'বৃহস্পতিবার',
  '০৮',
  'জুলাই',
  '২০১০',
  '০২২৪৩৩',
  'পরিচ'],
 ['',
  '',
  'ডেঙ্গু',
  '',
  'ডোঙ্গল',
  'লিখেছেন',
  'আশাবাদী',
  'পোস্টটি',
  'করা',
  'হয়েছে',
  'শনিবার',
  '১০',
  'জুলাই',
  '২০১০',
  '২৩২৪৫৯',
  'ক্লাসিক্যাল',
  'ডেঙ্'],
 ['', '', 'আবার', '', 'ডেঙ্গু', 'জ্বর', '', 'সতর্ক', 'থাকুন', '', ''],
 ['24',
  '',
  'ডেঙ্গু',
  '',
  'আক্রান্ত',
  'যুবরাজকে',
  'ছাড়াই',
  'নামছে',
  'ভারত',
  ''],
 ['',
  '',
  '',
  'ডেঙ্গু',
  'জ্বরের',
  'সেবাযত্ন',
  'ডেঙ্গু',
  'কি',
  '১ডেঙ্গু',
  'একটি',
  'ভাইরাসজনিত',
  'জ্বর',
  'এডিস',
  'মশার',
  'মাধ্যমে',
  '',
  'ডেঙ্গু',
  'জ্বর',
  '',
  'ছড়ায়'],
 ['',
  '',
  '',
  'চারিদিকে',
  'প্রচুর',
  '',
  'ডেঙ্গু',
  '',
  'জ্বরের',
  'প্রকোপ',
  'বেড়েছে',
  '',
  'ডেঙ্গু',
  '',
  'কি',
  '১ডেঙ্গু',
  '',
  'একটি',
  'ভাইরাসজনিত',
  'জ্বর',
  'এডিস',
  'মশার',
  'মাধ্'],
 ['', '', '', '', 'ডেঙ্গু', ''],
 ['',
  '',
  '',
 

In [61]:
for i in range(len(tokenized_data)):
    while '' in tokenized_data[i]:
        tokenized_data[i].remove('')

In [63]:
data_words = tokenized_data.copy()

In [68]:
data_words

[['ডেঙ্গু',
  'ডোঙ্গল',
  'লিখেছেন',
  'পোস্টটি',
  'করা',
  'হয়েছে',
  'বৃহস্পতিবার',
  '০৮',
  'জুলাই',
  '২০১০',
  '০২২৪৩৩',
  'পরিচ'],
 ['ডেঙ্গু',
  'ডোঙ্গল',
  'লিখেছেন',
  'আশাবাদী',
  'পোস্টটি',
  'করা',
  'হয়েছে',
  'শনিবার',
  '১০',
  'জুলাই',
  '২০১০',
  '২৩২৪৫৯',
  'ক্লাসিক্যাল',
  'ডেঙ্'],
 ['আবার', 'ডেঙ্গু', 'জ্বর', 'সতর্ক', 'থাকুন'],
 ['24', 'ডেঙ্গু', 'আক্রান্ত', 'যুবরাজকে', 'ছাড়াই', 'নামছে', 'ভারত'],
 ['ডেঙ্গু',
  'জ্বরের',
  'সেবাযত্ন',
  'ডেঙ্গু',
  'কি',
  '১ডেঙ্গু',
  'একটি',
  'ভাইরাসজনিত',
  'জ্বর',
  'এডিস',
  'মশার',
  'মাধ্যমে',
  'ডেঙ্গু',
  'জ্বর',
  'ছড়ায়'],
 ['চারিদিকে',
  'প্রচুর',
  'ডেঙ্গু',
  'জ্বরের',
  'প্রকোপ',
  'বেড়েছে',
  'ডেঙ্গু',
  'কি',
  '১ডেঙ্গু',
  'একটি',
  'ভাইরাসজনিত',
  'জ্বর',
  'এডিস',
  'মশার',
  'মাধ্'],
 ['ডেঙ্গু'],
 ['মশা',
  'দিয়ে',
  'মশা',
  'নিধন',
  'স্ত্রী',
  'এডিস',
  'মশার',
  'কামড়েই',
  'ডেঙ্গু',
  'জ্বর',
  'হয়ে',
  'থাকে',
  'বিশ্ব',
  'স্বাস্থ্য',
  'সংস্থার',
  'হিসাবে',
  'ব'],
 ['ডেঙ্গু',
  'জ্বর',
  'রুখতে',
  'ব

In [64]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['ডেঙ্গু', 'ডোঙ্গল', 'লিখেছেন', 'পোস্টটি', 'করা', 'হয়েছে', 'বৃহস্পতিবার', '০৮', 'জুলাই', '২০১০', '০২২৪৩৩', 'পরিচ']


In [66]:

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [67]:

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:10])

[['ডেঙ্গু', 'ডোঙ্গল', 'লিখেছেন', 'হয়েছে', 'পরিচ'], ['ডেঙ্গু', 'আশাবাদী', 'হয়েছে'], ['ডেঙ্গু', 'জ্বর'], ['ডেঙ্গু', 'আক্রান্ত', 'যুবরাজকে', 'নামছে', 'ভারত'], ['ডেঙ্গু', 'সেবাযত্ন', 'ডেঙ্গু', 'একটি', 'ভাইরাসজনিত', 'জ্বর', 'এডিস', 'ডেঙ্গু', 'জ্বর'], ['ডেঙ্গু', 'বেড়েছে', 'ডেঙ্গু', 'একটি', 'ভাইরাসজনিত', 'জ্বর', 'এডিস', 'মাধ্'], ['ডেঙ্গু'], ['স্ত্রী', 'এডিস', 'জ্বর', 'হয়ে', 'সংস্থার', 'হিসাবে'], ['ডেঙ্গু', 'বিজ্ঞানীদের', 'সাফল্য', 'নানা', 'খবর'], ['ডেঙ্গু', 'বিজ্ঞানীদের', 'সাফল্য', 'নানা', 'খবর']]


In [69]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:10])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(0, 1), (4, 1), (5, 1)], [(0, 1), (6, 1)], [(0, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(0, 3), (6, 2), (11, 1), (12, 1), (13, 1), (14, 1)], [(0, 2), (6, 1), (11, 1), (12, 1), (13, 1), (15, 1), (16, 1)], [(0, 1)], [(6, 1), (12, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(0, 1), (21, 1), (22, 1), (23, 1), (24, 1)], [(0, 1), (21, 1), (22, 1), (23, 1), (24, 1)]]


In [70]:
id2word[0]

'ডেঙ্গু'

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
import os

In [ ]:
os.environ['MALLET_HOME'] = 'C:\\mallet'

mallet_path = 'C:\\mallet\\bin\\mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
model_list

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()